In [ ]:
import pandas as pd
import os
import glob
from pathlib import Path

In [ ]:
def file_key(file):
    base_name = os.path.basename(file)
    year_month = base_name.split('-citibike-tripdata')[0]
    year, month = year_month[:4], year_month[4:]
    return int(year) * 100 + int(month)

In [ ]:
data_path = Path('./../data/citibike')

# Get all the CSV files in the citibike data directory
csv_files = glob.glob(str(data_path / '*.csv'))

# Sort the CSV files in chronological order
csv_files.sort(key=file_key)

In [ ]:
# Dictionary to store the grouped CSV files
grouped_files = {}

# Loop through all CSV files and read their columns
for file in csv_files:
    print(f"Processing {file}")
    data = pd.read_csv(file, nrows=0)  # Read only the columns, no rows
    columns = tuple(data.columns)

    # Add the file to the appropriate group
    if columns in grouped_files:
        grouped_files[columns].append(file)
    else:
        grouped_files[columns] = [file]

print("\nGrouped files:")
for i, columns in enumerate(grouped_files.keys(), start=1):
    print(f"Group {i}: {list(columns)}")

In [ ]:
# Print the content of dictionary
for columns, files in grouped_files.items():
    print(f"Group {columns}:")
    for file in files:
        print(f"  {file}")

In [ ]:
def find_largest_subgroup(files):
    largest_subgroup = []
    current_subgroup = [files[0]]

    for prev, current in zip(files[:-1], files[1:]):
        prev_key = file_key(prev)
        current_key = file_key(current)

        # Check if consecutive files are 1 month apart or a new year begins
        if current_key - prev_key == 1 or (current_key % 100 == 1 and prev_key % 100 == 12):
            current_subgroup.append(current)
        else:
            # Compare current_subgroup and largest_subgroup, update if needed
            if len(current_subgroup) > len(largest_subgroup):
                largest_subgroup = current_subgroup

            # Reset current_subgroup
            current_subgroup = [current]

    # Final comparison for the last subgroup
    if len(current_subgroup) > len(largest_subgroup):
        largest_subgroup = current_subgroup

    return largest_subgroup

print("\nLargest subgroups with no missing months:")
for i, (columns, files) in enumerate(grouped_files.items(), start=1):
    largest_subgroup = find_largest_subgroup(files)
    print(f"Group {i}: {list(columns)}")
    print("  Largest subgroup length:", len(largest_subgroup))
    print("  Largest subgroup:", largest_subgroup)


# Choosing the subgroup
Let us choose the data from 2021-02 to 2023-03

In [ ]:
# Find index of selected data in csv_files
start_index = csv_files.index(str(data_path / '202102-citibike-tripdata.csv'))
end_index = csv_files.index(str(data_path / '202303-citibike-tripdata.csv'))
selected_files = csv_files[start_index:end_index+1]

In [ ]:
# Initialize an empty list to store DataFrames
dataframes = []

# Read each CSV file and append the DataFrame to the list
for file in selected_files:
    df = pd.read_csv(file)
    dataframes.append(df)

# Concatenate all the DataFrames
merged_dataframe = pd.concat(dataframes, ignore_index=True)

In [ ]:
# Extract YYYYMM from the first and last file in selected_files
start_YYYYMM = selected_files[0].split('/')[-1][:6]
end_YYYYMM = selected_files[-1].split('/')[-1][:6]

# Create the output file name
output_file_name = f"{start_YYYYMM}-{end_YYYYMM}-citibike-tripdata-merged.csv"
output_file = data_path / output_file_name

In [ ]:
# Write the concatenated DataFrame to the output CSV file
merged_dataframe.to_csv(output_file, index=False)